# Função responsável por gerar a chave através de uma password

A geração da chave é realizada com recurso a uma **KDF**, nomeadamente, a `PBKDF2HMAC`, que se encontra disponível no módulo **Cryptography**.

In [1]:
import os
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac

def generate_key(password, salt=os.urandom(16)):
    backend = default_backend()
    
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=64,
        salt=salt,
        iterations=100000,
        backend=backend
    )

    key = kdf.derive(password)

    return key

ImportError: No module named cryptography.hazmat.primitives.kdf.pbkdf2

# Função responsável por gerar o MAC de um criptograma

Para o efeito foi utilizado o `HMAC_SHA256` e a chave gerada para produzir um MAC que identifica unicamente o criptograma em questão. Por um lado, o emissor utiliza esta função para produzir o MAC que é enviado juntamente com o criptograma. Por outro lado, o recetor utiliza-a com o objetivo de verificar o MAC associado ao criptograma recebido e, consequentemente, verificar a integridade da mensagem.

In [ ]:
def generate_mac(key, crypto):
    h = hmac.HMAC(key, hashes.SHA256(), backend = default_backend())
    h.update(crypto)
    return h.finalize()

# Função responsável por cifrar a mensagem a ser enviada

Para cifrar a mensagem a ser enviada é utilizado a cifra simétrica **AES** no modo **GCM**. 

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def encrypt(plaintext, key, associated_data):

    # Generate a random 96-bit IV.
    iv = os.urandom(12)
    
    # Construct an AES-GCM Cipher object with the given key and a randomly generated IV.
    encryptor = Cipher(algorithms.AES(key[:32]), modes.GCM(iv), backend=default_backend()).encryptor()
    
    # associated_data will be authenticated but not encrypted, it must also be passed in on decryption.
    encryptor.authenticate_additional_data(associated_data)

    # Encrypt the plaintext and get the associated ciphertext. GCM does not require padding.
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()

    package = iv + encryptor.tag + ciphertext

    hmac = generate_mac(key[32:], package)

    return hmac + package

# Função responsável por decifrar a mensagem recebida



In [ ]:
def decrypt(package, key, associated_data):
    
    hmac = package[:32]

    macDest = generate_mac(key[32:], package[32:])

    if (hmac != macDest):
        return 'ERROR - MAC/Password is not equal'

    iv = package[32:44]

    tag = package[44:60]

    ciphertext = package[60:]
    
    # Construct a Cipher object, with the key, iv, and additionally the GCM tag used for authenticating the message.
    decryptor = Cipher(algorithms.AES(key[:32]), modes.GCM(iv, tag), backend=default_backend()).decryptor()

    # We put associated_data back in or the tag will fail to verify when we finalize the decryptor.
    decryptor.authenticate_additional_data(associated_data)

    # Decryption gets us the authenticated plaintext. If the tag does not match 
    # an InvalidTag exception will be raised.
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    return plaintext.decode()

# Processos que permitem a comunicação privada assíncrona entre um agente Emitter e um agente Receiver

In [ ]:
from multiprocessing import Process, Pipe
from getpass import getpass
from base64 import b64encode, b64decode
import time
import lorem

my_salt = os.urandom(16)

def Emitter(conn):
    passwd = getpass('Emmiter password: ').encode('utf-8') 
    key = generate_key(passwd, my_salt) # geração da chave a partir da password
    
    mess = lorem.sentence()
    print('Mensagem inicial: ' + mess)
    crypto = encrypt(mess, key, my_salt)
    #print('Criptograma enviado: ' + b64encode(crypto).decode())
    conn.send(crypto)  # envia uma mensagem pelo seu lado do Pipe
    conn.close()        # termina a ligação do seu lado do Pipe
    
def Receiver(conn):
    passwd = getpass('Emmiter password: ').encode('utf-8')   
    key = generate_key(passwd, my_salt) # geração da chave a partir da password
    
    crypto = conn.recv()  # recebe a mensagem do seu lado do Pipe
    #print('Criptograma recebido: ' + b64encode(crypto).decode())
    mess = decrypt(crypto, key, my_salt)
    print('Mensagem recebida: ' + mess)         # faz qualquer coisa com a informação recebida
    conn.close()        # fecha a ligação do seu lado

class BiConn(object):
    def __init__(self, emitter, receiver, timeout=None):
        """
        emitter : a função que vai ligar ao lado esquerdo do Pipe
        receiver: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        emitter_end, receiver_end = Pipe()
        self.timeout = timeout        
        self.eproc = Process(target=emitter, args=(emitter_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=receiver, args=(receiver_end,))
        self.emitter  = lambda : emitter(emitter_end)                       # as funções ligadas já ao Pipe
        self.receiver = lambda : receiver(receiver_end)
    
    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.eproc.start()
            self.rproc.start()
            self.eproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()
    
    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.emitter()
        self.receiver()
    
    
Conn = BiConn(Emitter, Receiver)
Conn.manual()